In [2]:
# Importare le librerie
import tweepy
from dotenv import dotenv_values
# Importare le variabili d'ambiente dal file .env
config = dotenv_values(".env")

In [3]:
# %load utils.py
import json
import os


def serealize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder,exist_ok=True)
    with open(f"{folder}/{filename}.json", "w", encoding="utf-8") as f:
        json.dump(data,f,ensure_ascii=False,indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}.json")

def read_json(path):
    try:
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as file:
                data = json.load(file)
            return data
    except ValueError:
        print("Path not found, check the correctness of the path")

## Punto 1

Creiamo un client Tweepy utilizzando il `BREARER_TOKEN` presente nel file `.env` e caricato tramite la libreria `dotenv`.

In [4]:
# Nuovo client Tweepy
client = tweepy.Client(bearer_token=config["BEARER_TOKEN"])

Utilizzando la funzione `get_user()` inviamo la richiesta per ricavare le informazioni di default, aggiungiamo inoltre le opportune `user_fields` per ricavare ulteriori informazioni dell'utente, come descrizione, stato di protezione, metriche pubbliche.  
Ricaviamo l'utente `kevin` dal corpo della risposta.

In [5]:
# Fetching dell'utente @KevinRoitero
response = client.get_user(username="KevinRoitero",user_fields=["description", "protected", "public_metrics"])
kevin = response.data

Convertiamo l'utente `kevin` in un dizionario strutturato per permettere la serializzazione in un file JSON.

In [6]:
# Serializzare i risultati in JSON
kevin_dict = {
    "id": kevin.id,
    "name": kevin.name,
    "username": kevin.username,
    "description": kevin.description,
    "public_metrics": kevin.public_metrics,
    "protected": kevin.protected
}
serealize_json("out",kevin.name,kevin_dict)

Data serialized to path: out/Kevin Roitero.json


## Punto 2

Tramite la funzione `get_users_followers()` ricavo la lista dei follower che seguono l'utente `kevin`. Successivamente per ogni follower, tramite `get_recent_tweets_count()`, ricavo, con scadenza giornaliera, il numero dei tweets recenti (negli ultimi 7 giorni). Per ricavare un valore unico settimanale, sommo i valori giornalieri in una variabile denominata `last_week_tweets_count`. Infine creo una directory temporanea, rappresentativa del follower, aggiundo varie informazioni riguardante esso, come: id, nome, username e il numero di tweets recenti (calcolato precedentemente). Aggiungo la directory ad una lista precedentemente inizializzata e successivamente aggiunto tale lista all'interno della directory principale `kevin_dict` all'interno della voce `followers`.

In [7]:
# Per ogni follower del profilo ricavare il numero di tweet pubblicati da quel profilo durante l'ultima settimana
response = client.get_users_followers(id=kevin.id)
followers = response.data
followers_list = []
for follower in followers:
    last_week_tweets_count = 0
    recent_tweets_count = client.get_recent_tweets_count(query=f"from:{follower.username} -is:retweet",granularity="day")
    for day in recent_tweets_count.data or []:
            last_week_tweets_count += day["tweet_count"]
    dummy_follower_dict = {
        "id": follower.id,
        "name": follower.name,
        "username": follower.username,
        "last_week_tweets_count": last_week_tweets_count
    }
    followers_list.append(dummy_follower_dict)
# Aggiungere il numero di tweets nel JSON
kevin_dict["followers"] = followers_list
serealize_json("out","Kevin with followers", kevin_dict)

Data serialized to path: out/Kevin with followers.json
